In [1]:
import pandas as pd
import seaborn as sns
import traintools
from trainconstants import *
from collections import defaultdict
import networkx as nx

sns.set()
%matplotlib inline

df = pd.read_csv("datasets/nsdata1.txt",  delimiter=r"\s+")
df['departure_time'] = df['departure_time'].map(traintools.to_minutes_past_midnight)
df['arrival_time'] = df['arrival_time'].map(traintools.to_minutes_past_midnight)
df.head()

,train_number,from,to,departure_time,arrival_time,first_class,second_class
0,2123,2,3,420,460,4,58
1,2123,3,4,463,518,14,328
2,2127,1,2,408,475,47,340
3,2127,2,3,481,521,35,272
4,2127,3,4,523,578,19,181


In [13]:
minimum_number_of_type_3_trains = traintools.minimum_number_of_trains_at_time_t(df, TYPE_3_TRAIN)
rush_hour_peak = int(minimum_number_of_type_3_trains.idxmax())
minimum_number_of_type_3_trains.loc[rush_hour_peak]

number of trains    22.0
Name: 1055, dtype: float64

In [14]:
# creating the new "loop" in the schedule
start = minimum_number_of_type_3_trains.ne(0).idxmax()

minimum_number_of_type_3_trains['time'] = minimum_number_of_type_3_trains.index

tail = minimum_number_of_type_3_trains.loc[rush_hour_peak:].copy()
head = minimum_number_of_type_3_trains.loc[start[0]:rush_hour_peak].copy()
head['time'] = head['time'].map(lambda x: x + 1440)

new_loop = pd.concat([tail, head])
new_loop['time'] = new_loop['time'].map(lambda x:  x - rush_hour_peak)
new_loop.head()

# Do this logic but let new_loop also have the actual number of passengers stored somewhere

,number of trains,time
1055,22.0,0
1056,22.0,1
1057,22.0,2
1058,22.0,3
1059,18.0,4


In [5]:
schedule_network = pd.DataFrame()
schedule_network['from'] = list(zip(df.from, df.departure_time))
schedule_network['to'] = list(zip(df.to, df.arrival_time))

G = nx.from_pandas_edgelist(df=schedule_network, source='from', target='to')#, edge_attr=['first_class', 'second_class'])

In [6]:
G.number_of_edges()

2